### This script analysis the ERSD response of all channels.

### Trial lengths range from 5.8-6.2s, so use a trial length of 6s;

### Baseline at each trial is 0.7s. Potential issue: there should be some response before 0.7s when subjects are prepared to do task.

Issue: Found no difference in TF representation between baseline and task.

Possible reason: Baseline is too short, only 0.7s.

In [22]:
from speech_pinyin.config import *
import glob
import matplotlib.pylab as plt
import mne
import numpy as np
from utils.util_MNE import keep_annotation
from mne.time_frequency import tfr_morlet


In [3]:
sid=1
folder=data_dir+str(sid)+'-*'
folder=os.path.normpath(glob.glob(folder)[0])
folder=folder.replace("\\", "/")

In [4]:
session=3
#filename=folder+'/processed/session'+str(session)+'.fif'
filename=folder+'/processed/session3_trigger_added-raw.fif'
raw=mne.io.read_raw_fif(filename)

In [8]:
annotations=raw.annotations
annotations=keep_annotation(annotations,['5'])
raw.set_annotations(annotations)

Measurement date,"August 28, 2024 08:33:34 GMT"
Experimenter,Unknown
Participant,X
Digitized points,Not available
Good channels,"120 EEG, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.50 Hz
Lowpass,495.00 Hz


In [16]:
events = mne.events_from_annotations(raw)[0] # 144
epochs = mne.Epochs(raw, events, tmin=0, tmax=6,baseline=None)
chnNum=len(epochs.ch_names)

In [20]:
decim=4
new_fs=1000/decim
n_cycles_mthod='stage' # or: equal
vmin=-4
vmax=4
fMin,fMax=2,150
fstep=1
freqs=np.arange(fMin,fMax,fstep) #148
fNum=freqs.shape[0]
#freqs = np.linspace(fMin,fMax, num=fNum)
cycleMin,cycleMax=8,50
cycleNum=fNum
#n_cycles = np.linspace(cycleMin,cycleMax, num=cycleNum)  # different number of cycle per frequency
groups=5
rates=[2,2.5,3,4,5]
num_per_group=int(fNum/groups)
if n_cycles_mthod=='equal':
    n_cycles=freqs
elif n_cycles_mthod=='stage':
    n_cycles=[]
    for g in range(groups):
        if g < groups -1:
            tmp=[int(i) for i in freqs[g*num_per_group:(g+1)*num_per_group]/rates[g]]
        elif g==groups -1:
            tmp = [int(i) for i in freqs[g * num_per_group:] / rates[g]]
        n_cycles.extend(tmp)


In [ ]:
erd_wind=10
ers_wind=2
erd_end_f=30
ers_start_f=30
erds_change = []
ch_power_avg = []

In [55]:
chid=0
chname=epochs.ch_names[chid]

In [74]:
baseline = [int(0.1*new_fs), int(0.7*new_fs)]
taskline=[int(4*new_fs), int(9*new_fs)]

In [75]:
# decim will decrease the sfreq, so 15s will becomes 5s afterward.

%matplotlib inline
chid=0
for chid in range(chnNum-1):
    fig,ax=plt.subplots()
    chname=epochs.ch_names[chid]
    # The very big artifact at begainning and end of the TF will cause ERS vanish after normalization.
    ch_power=np.squeeze(tfr_morlet(epochs, picks=[chname], freqs=freqs, n_cycles=n_cycles,use_fft=True,return_itc=False, 
                                   average=False, decim=decim, n_jobs=1).data) # (144, 148, 1501) (trials, frequency, time)
    base = ch_power[:,:, baseline[0]:baseline[1]] #  (144, 148, 175)
    basemean = np.mean(base, 2) # (144, 148)
    basestd= np.std(base, 2) #(144, 148) 
    ch_power_norm = 10*np.log10(ch_power / basemean[:, :, None]) #
    ax.imshow(ch_power_norm[1,:,:],origin='lower', cmap='RdBu_r') # same as below
    filename=folder+'/processed/tf_result/session3_ch'+str(chid)+'.png'
    fig.savefig(filename, dpi=400)
    fig.clear()
    plt.close(fig)
    

All plots are similar to this plot:

![](img\7.png)

On gesture dataset, this method can produce below TF plot:

![](img\6.png)

### Below using methods from MNE

However, the result is not good, as shown in below. No difference in TF representation between baseline and task.

In [ ]:
%matplotlib inline
chid=0

for chid in range(chnNum-1):
    fig,ax=plt.subplots()
    chname=epochs.ch_names[chid]
    tfr_epochs=epochs.load_data().copy().pick(picks=[chname]).compute_tfr("morlet",freqs,n_cycles=n_cycles,decim=decim,average=False,return_itc=False,n_jobs=None,)
    tfr_epochs.apply_baseline(mode="logratio", baseline=(0, 0.7)) # (144, 1, 148, 1501)
    tf_data=tfr_epochs.data
    ax.imshow(tf_data[0,0,:,:],origin='lower', cmap='RdBu_r')
    filename=folder+'/processed/tf_result/session3_ch'+str(chid)+'.png'
    fig.savefig(filename, dpi=400)
    fig.clear()
    plt.close(fig)

No difference in below result.

![](img\5.png)

In [47]:
chnNum

121